In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from qutip.qip.circuit import QubitCircuit, Gate, gate_sequence_product, Measurement, CircuitSimulator

qutip.settings.has_mkl = False  # Fix the OS erros in VSCode

# Gates

## IBM Q Gates

In [2]:
def U3(args):
    theta, phi, lambda_ = args
    matrix = [
        [np.cos(theta / 2), -np.exp(1j * lambda_) * np.sin(theta / 2)],
        [np.exp(1j * phi) * np.sin(theta / 2), np.exp(1j * lambda_ + 1j * phi) * np.cos(theta / 2)],
    ]
    return Qobj(matrix, dims=[[2], [2]])


In [3]:
def U2(args):
    phi, lambda_ = args
    return U3([np.pi/2, phi, lambda_])

In [4]:
def U1(lambda_):
    return U3([0,0,lambda_])

In [5]:
def CU3(args):
    mat = np.zeros((4, 4), dtype=complex)
    mat[0, 0] = mat[1, 1] = 1.0
    mat[2:4, 2:4] = U3(args)

    return Qobj(mat, dims=[[2, 2], [2, 2]])

CU3([1, 0, 0])


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.87758256 -0.47942554]
 [ 0.          0.          0.47942554  0.87758256]]

## Unitary Evo

In [6]:
def U(args):
    delta, epsilon, dt = args
    Rz = (1j * delta * sigmaz() * dt / 2).expm()
    Ry = (1j * epsilon * sigmay() * dt / 2).expm()
    return Rz * Ry * Rz

# Circuit

In [31]:
dt = 1

delta = 1
epsilon = 1

Gamma = 1
theta = 2*np.arcsin(np.sqrt(Gamma * dt))

In [8]:
qc = QubitCircuit(2, user_gates={"U3": U3, "CU3": CU3, "Uevo": U}, num_cbits=1)

## Unitary Gates

In [9]:
qc.add_gate("Uevo", arg_value=[delta, epsilon, dt])

## Relaxation

In [10]:
def reset(reg, bit, new):
    """Reset the qubit at position bit of reg with dm new"""
    traces = [reg.ptrace(i) for i in range(int(np.log2(reg.shape[0])))]

    traces[bit] = ket2dm(new) if new.type == "ket" else new
    return tensor(traces)


In [11]:
qc.add_gate("CU3", arg_value=[theta, 0, 0])
qc.add_gate("CNOT", targets=[0], controls=[1])
qc.add_measurement("M", [1], classical_store=0)

In [12]:
p = qc.propagators()

# Doesn't include the measurement or reset
D = gate_sequence_product(p)
D

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.99376067+0.09970865j  0.        +0.j          0.04997917+0.j
   0.        +0.j        ]
 [-0.0158048 +0.j         -0.0474144 +0.j          0.31425472-0.03153064j
   0.94276415-0.09459193j]
 [-0.0474144 +0.j          0.0158048 +0.j          0.94276415-0.09459193j
  -0.31425472+0.03153064j]
 [ 0.        +0.j          0.99376067+0.09970865j  0.        +0.j
   0.04997917+0.j        ]]

# Simulation

In [36]:
initial_state = tensor(basis(2, 0), basis(2, 0))

b = Bloch3d()
b.add_states(initial_state.ptrace(0))

# sim = CircuitSimulator(qc, mode="density_matrix_simulator")

# final = sim.run(initial_state).get_final_states()[0]
# b.add_states(final.ptrace(0))

# for _ in range(10):
#     final = sim.run(final).get_final_states()[0]
#     b.add_states(final.ptrace(0))


for _ in range(40):
    result = qc.run(state=initial_state)
    reset_result = reset(result, 1, basis(2,0))

    b.add_states(reset_result.ptrace(0))

    initial_state = reset_result

b.show()
